In [ ]:
import pandas as pd


df = pd.read_csv("../data/raw/MachineLearningRating_v3.txt")

df.head()
# df.info()


FileNotFoundError: [Errno 2] No such file or directory: 'data/raw/MachineLearningRating_v3.txt'

In [ ]:
missing=df.isnull().sum().sort_values(ascending=False)
missing_percent=(df.isnull().mean()*100).sort_values(ascending=False)
pd.concat([missing,missing_percent],axis=1,keys=["n_missing","pct_missing"]).head(30)

,n_missing,pct_missing
age,0,0.0
sex,0,0.0
bmi,0,0.0
children,0,0.0
smoker,0,0.0
region,0,0.0
charges,0,0.0


In [ ]:
df.duplicated(subset=["PolicyID"]).sum()
cat_cols = ["Province","VehicleType","Gender","Make","Model","PostalCode","CoverType"]
for c in cat_cols:
    df[c] = df[c].astype("category")



KeyError: Index(['PolicyID'], dtype='object')

In [ ]:
df["LossRatio"] = df["TotalClaims"] / df["TotalPremium"]
# Claim frequency: policy-level indicator (1 if any claim)
df["HasClaim"] = (df["TotalClaims"] > 0).astype(int)
# Claim severity conditional on claim
df_claims = df[df["HasClaim"]==1].copy()
df_claims["ClaimSeverity"] = df_claims["TotalClaims"]  # rename for clarity
# Margin
df["Margin"] = df["TotalPremium"] - df["TotalClaims"]


In [ ]:
num_cols = ["TotalPremium","TotalClaims","CustomValueEstimate","CalculatedPremiumPerTerm","LossRatio"]
df[num_cols].describe().T

# Additional variability metrics:
df[num_cols].agg(["mean","median","std","var","min","max","skew","kurtosis"]).T
# Interquartile range
iqr = df[num_cols].quantile(0.75) - df[num_cols].quantile(0.25)
iqr


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

for col in ["TotalClaims","TotalPremium","CustomValueEstimate"]:
    plt.figure(figsize=(8,4))
    sns.histplot(df[col].dropna(), bins=60)
    plt.title(col)
    plt.show()

    # Log plot if skewed (add 1 to handle zeros)
    plt.figure(figsize=(8,4))
    sns.histplot(np.log1p(df[col].dropna()), bins=60)
    plt.title(f"log1p({col})")
    plt.show()


In [ ]:
plt.figure(figsize=(10,4))
sns.boxplot(x=df["TotalClaims"])
plt.title("TotalClaims boxplot")
plt.show()


In [ ]:
for col in ["Province","VehicleType","Gender"]:
    grp = df.groupby(col)["LossRatio"].agg(["mean","median","count","std"]).sort_values("mean", ascending=False)
    display(grp.head(15))


In [ ]:
plt.figure(figsize=(12,6))
sns.barplot(x="Province", y="LossRatio", data=df, estimator=np.mean)
plt.xticks(rotation=45)
plt.title("Average LossRatio by Province")
plt.show()


In [ ]:
freq = df.groupby("Province")["HasClaim"].mean().sort_values(ascending=False)
sev = df_claims.groupby("Province")["ClaimSeverity"].mean().sort_values(ascending=False)
pd.concat([freq, sev], axis=1, keys=["ClaimFreq","AvgSeverity"]).head(15)


In [ ]:
ts = df.set_index("TransactionMonth").resample("M").agg({
    "TotalClaims":"sum",
    "TotalPremium":"sum",
    "HasClaim":"mean"   # frequency as proportion per month
})
ts["LossRatio"] = ts["TotalClaims"] / ts["TotalPremium"]
ts.plot(subplots=True, figsize=(10,8), title="Monthly trends")


In [ ]:
ts_pct = ts.pct_change().dropna()


In [ ]:
pc = df.groupby("PostalCode").agg({
    "TotalPremium":"sum",
    "TotalClaims":"sum",
    "HasClaim":"mean"
}).reset_index()
pc["LossRatio"] = pc["TotalClaims"]/pc["TotalPremium"]

# Scatter: premium vs claims
plt.figure(figsize=(8,6))
sns.scatterplot(data=pc.sample(500), x="TotalPremium", y="TotalClaims", size="HasClaim", alpha=0.6)
plt.xscale("log"); plt.yscale("log")
plt.title("PostalCode: TotalPremium vs TotalClaims (sample)")
plt.show()


In [ ]:
mm = df.groupby(["Make","Model"]).agg({
    "TotalClaims":"mean",
    "HasClaim":"mean",
    "TotalPremium":"mean",
    "LossRatio":"mean",
    "PolicyID":"count"
}).rename(columns={"PolicyID":"n_policies"}).sort_values("TotalClaims", ascending=False)
mm.head(20)


In [ ]:
mm_filtered = mm[mm["n_policies"] >= 30]  # threshold
mm_filtered.sort_values("LossRatio", ascending=False).head(20)


In [ ]:
num = df[["TotalPremium","TotalClaims","CustomValueEstimate","LossRatio"]].copy()
corr = num.corr()
plt.figure(figsize=(6,4))
sns.heatmap(corr, annot=True, fmt=".2f")
plt.title("Correlation matrix")
plt.show()


In [ ]:
def iqr_outliers(series):
    q1, q3 = series.quantile(0.25), series.quantile(0.75)
    iqr = q3 - q1
    lower, upper = q1 - 1.5*iqr, q3 + 1.5*iqr
    return series[(series < lower) | (series > upper)]

outliers_claims = iqr_outliers(df["TotalClaims"].dropna())
len(outliers_claims)


In [ ]:
# 1. Faceted barplot (Province × top VehicleTypes)
top_types = df["VehicleType"].value_counts().nlargest(6).index
sub = df[df["VehicleType"].isin(top_types)]
g = sns.catplot(data=sub, x="Province", y="LossRatio", col="VehicleType",
                kind="bar", col_wrap=3, height=3, aspect=1.6, sharey=True)
g.set_xticklabels(rotation=45)


In [ ]:
# 2. Scatter with marginal histograms (seaborn jointplot won't handle size easily, so use scatter + hist)
agg_pc = df.groupby("PostalCode").agg({
    "TotalPremium":"sum","TotalClaims":"sum","PolicyID":"count"
}).rename(columns={"PolicyID":"n_policies"})
agg_pc["LossRatio"] = agg_pc["TotalClaims"]/agg_pc["TotalPremium"]
samp = agg_pc.sample(1000).reset_index()
plt.figure(figsize=(8,6))
plt.scatter(samp["TotalPremium"], samp["TotalClaims"], 
            s=(samp["n_policies"]/samp["n_policies"].max())*200, # size
            c=samp["LossRatio"], alpha=0.6)
plt.xscale("log"); plt.yscale("log")
plt.colorbar(label="LossRatio")
plt.xlabel("TotalPremium (log)")
plt.ylabel("TotalClaims (log)")
plt.title("PostalCode aggregated premium vs claims (sample)")
plt.show()
